# Parameter extraction from Parlogs-observations dataset

The instructions below are based on https://github.com/paranal-sw/eliana/blob/main/notebooks/02-datasets.ipynb and the possible configurations are listed in https://huggingface.co/datasets/Paranal/parlogs-observations

## Configure the dataset

In [1]:
# PIONIER MATISSE GRAVITY
INSTRUMENT='MATISSE'

# 1d 1w 1m 6m
RANGE='1d'

In [2]:
import pandas as pd
import numpy as np
from eliana.datasets import ParlogsObservations
logs = ParlogsObservations(period=RANGE, source="Instrument", system=INSTRUMENT)

## Templates in the dataset
The *Templates* are atomic units of observations. They are the smallest unit of observation that can be executed by the telescope. They are defined by a set of parameters that are used to configure the telescope and the instruments. Each template correspond to a script in Tcl/Tk language, identified by a unique TPL_ID, and is related to a single instrument.

In [3]:
# List the first 15 templates
print(f"Number of templates: {len(logs.meta)}")

logs.meta[0:15]


Number of templates: 70


,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,USER_ABORT,SECONDS,TEL,TPL_EXEC
0,2019-04-10 00:16:03.269,2019-04-10 00:20:35.831,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,272.0,AT,STOP
1,2019-04-10 00:20:35.848,2019-04-10 00:42:17.085,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,1301.0,AT,STOP
2,2019-04-10 00:42:17.106,2019-04-10 01:11:09.300,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,1732.0,AT,STOP
3,2019-04-10 01:11:09.325,2019-04-10 01:22:27.609,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,678.0,AT,STOP
4,2019-04-10 01:22:40.579,2019-04-10 01:29:34.127,False,MATISSE,bob_ins,MATISSE_img_acq,False,False,413.0,AT,STOP
5,2019-04-10 01:29:34.143,2019-04-10 01:42:20.955,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,766.0,AT,STOP
6,2019-04-10 01:42:20.968,2019-04-10 02:05:36.905,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,1395.0,AT,STOP
7,2019-04-10 02:05:36.920,2019-04-10 02:27:25.100,False,MATISSE,bob_ins,MATISSE_hyb_obs,False,False,1308.0,AT,STOP
8,2019-04-10 02:28:30.917,2019-04-10 02:28:44.054,False,MATISSE,bob_ins,MATISSE_img_acq,False,True,13.0,AT,ABORT
9,2019-04-10 02:29:03.290,2019-04-10 02:34:37.571,False,MATISSE,bob_ins,MATISSE_img_acq,True,False,334.0,AT,ABORT


In [4]:
# List of different TPL_IDs in the dataframe logs.meta
TPL_IDs = logs.meta['TPL_ID'].value_counts()
TPL_IDs

TPL_ID
MATISSE_hyb_obs                  30
MATISSE_img_acq                  26
MATISSE_gen_cal_bias              4
MATISSE_gen_tec_opd               3
MATISSE_gen_cal_imbasic           2
errseverity                       2
MATISSE_gen_cal_transfunc         1
MATISSE_gen_tec_standby           1
MATISSE_gen_tec_check_fringes     1
Name: count, dtype: int64

## Extract the traces of one template

The execution of a template is called a *trace*. A trace produces a sequence of events that are recorded during the execution of the template. Each trace is identified by a unique trace_id, that corresponds to the id column in the dataframe logs.meta. The traces are stored in a dataframe that can be accessed using the `logs.load_trace` or combined in a single dataframe using the `logs.traces` method.

In [5]:
# Contents of a trace using load_trace
df = logs.load_trace(10)
df

,@timestamp,system,hostname,loghost,logtype,envname,procname,procid,module,keywname,keywvalue,keywmask,logtext,trace_id,event
0,2019-04-10 02:34:37.587,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,MATISSE_hyb_obs -- Celestial target observatio...,10,LOG bob_ins MATISSE_hyb_obs -- Celestial tar...
1,2019-04-10 02:34:37.587,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,Started at 2019-04-10T02:34:37 (underlined),10,LOG bob_ins Started at 2019-04-10T02:34:37 (...
2,2019-04-10 02:34:37.851,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,DET1 DIT = '0.111',10,LOG bob_ins DET1 DIT = '0.111'
3,2019-04-10 02:34:37.851,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,DET1 NCOHERENT VAL = '1.',10,LOG bob_ins DET1 NCOHERENT VAL = '1.'
4,2019-04-10 02:34:37.851,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,DET1 NINCOHERENT VAL = '1.',10,LOG bob_ins DET1 NINCOHERENT VAL = '1.'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4172,2019-04-10 03:01:37.139,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,-1 (SpringGreen4),10,LOG bob_ins -1 (SpringGreen4)
4173,2019-04-10 03:01:37.139,MATISSE,wmt,wmt,LOG,wmt,mtoControl,130,boss,,,,SETUP command done.,10,LOG mtoControl SETUP command done.
4174,2019-04-10 03:01:37.140,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,Template MATISSE_hyb_obs finished.,10,LOG bob_ins Template MATISSE_hyb_obs finished.
4175,2019-04-10 03:01:37.142,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,Finished in 1620 seconds at 2019-04-10T03:01:3...,10,LOG bob_ins Finished in 1620 seconds at 2019...


In [6]:
# An alternative method is to collect all traces and filter by trace_id
df_traces = logs.traces()
df_traces[df_traces['trace_id'] == 10]

,@timestamp,system,hostname,loghost,logtype,envname,procname,procid,module,keywname,keywvalue,keywmask,logtext,trace_id,event
21971,2019-04-10 02:34:37.587,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,MATISSE_hyb_obs -- Celestial target observatio...,10,LOG bob_ins MATISSE_hyb_obs -- Celestial tar...
21972,2019-04-10 02:34:37.587,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,Started at 2019-04-10T02:34:37 (underlined),10,LOG bob_ins Started at 2019-04-10T02:34:37 (...
21973,2019-04-10 02:34:37.851,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,DET1 DIT = '0.111',10,LOG bob_ins DET1 DIT = '0.111'
21974,2019-04-10 02:34:37.851,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,DET1 NCOHERENT VAL = '1.',10,LOG bob_ins DET1 NCOHERENT VAL = '1.'
21975,2019-04-10 02:34:37.851,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,DET1 NINCOHERENT VAL = '1.',10,LOG bob_ins DET1 NINCOHERENT VAL = '1.'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26143,2019-04-10 03:01:37.139,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,-1 (SpringGreen4),10,LOG bob_ins -1 (SpringGreen4)
26144,2019-04-10 03:01:37.139,MATISSE,wmt,wmt,LOG,wmt,mtoControl,130,boss,,,,SETUP command done.,10,LOG mtoControl SETUP command done.
26145,2019-04-10 03:01:37.140,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,Template MATISSE_hyb_obs finished.,10,LOG bob_ins Template MATISSE_hyb_obs finished.
26146,2019-04-10 03:01:37.142,MATISSE,wmt,wmt,LOG,wmt,bob_ins,155,seq,,,,Finished in 1620 seconds at 2019-04-10T03:01:3...,10,LOG bob_ins Finished in 1620 seconds at 2019...


## Extract parameters of one observation
Please note that this function can be improved.

In [7]:
def extract_params(df_original):
    # df is a copy fo df_original
    df = df_original.copy()
    
    # Filter rows where @timestamp are the first 3 seconds of the whole df
    df = df[df['@timestamp'] < df['@timestamp'].min() + pd.Timedelta(seconds=3)]
    
    # Filter where column logtext has the form (any string)' = '(anystring), and exactly one = sign
    df = df[df['logtext'].str.contains(r"^.* = .*$")] 
    
    # Create column "param" with left part of ' = ', and column "value" with right part of ' = '
    df['param'] = df['logtext'].str.split(' = ').str[0]
    df['value'] = df['logtext'].str.split(' = ').str[1]
    
    # Restrict to columns param and value
    df = df[['param', 'value']]
    
    # Replace spaces by dot in param
    df['param'] = df['param'].str.replace(" ", ".")

    # Remove extra ' enclosing value
    df['value'] = df['value'].str.replace("'", "")
    
    # Create a column numval with the numeric value of value, if possible, otherwise NaN
    df['numval'] = pd.to_numeric(df['value'], errors='coerce')
    
    # Create column strval if numval is NaN, otherwise empty string
    df['strval'] = df['value'].where(df['numval'].isna(), "")
    
    # Remove column value
    df = df.drop(columns=['value']) 

    # Set param as the index
    df = df.set_index('param')

    # # Add a first param named "@timestamp" where numval is NaN and strval is the first timestamp in df_original
    # # Create a new DataFrame with the new row
    # df = pd.concat( [pd.DataFrame({
    #     'numval': [None],
    #     'strval': [df_original['@timestamp'].iloc[0]]
    # }, index=['@timestamp']), df], axis=0 )
    df = df.sort_index()  # sorting by index
    
    return df

### Examples

Extract the parameters of trace=11

In [8]:
df = df_traces[df_traces['trace_id'] == 11]

# Allow to display all rows in Jupyter
with pd.option_context('display.max_rows', None):
    display(extract_params( df ))

,numval,strval
param,,
DET1.APOTYPE.VAL,0.00,
DET1.APOX.VAL,512.00,
DET1.APOY.VAL,512.00,
DET1.DIT,0.06,
DET1.INTERFX.VAL,512.00,
DET1.INTERFY.VAL,512.00,
DET1.NCOHERENT.VAL,1.00,
DET1.NINCOHERENT.VAL,1.00,
DET1.READ.CURNAME,NaN,SCI-SLOW-SPEED


## Combine parameters of same template (TPL_ID)

In [9]:
# Let's use one specific TPL_ID:
TPL_ID = TPL_IDs.index[0]
TPL_ID

'MATISSE_hyb_obs'

In [10]:
# Filter the meta dataframe to get the rows corresponding to TPL_ID
same_tpl_meta = logs.meta[logs.meta['TPL_ID'] == TPL_ID]

# Show the indices of the rows corrersponding to the same TPL_ID in the meta dataframe
same_tpl_meta.index

Index([ 1,  2,  3,  5,  6,  7, 10, 11, 12, 16, 17, 18, 20, 21, 22, 24, 25, 27,
       28, 29, 31, 32, 34, 35, 36, 38, 39, 54, 55, 56],
      dtype='int64')

In [11]:
# Creates a dataframe from traces with cols trace_id, @timestamp wehre @timestamp is the first in each trace_id group
df_timestamps = df_traces.groupby('trace_id').first().reset_index()[['trace_id', '@timestamp']]

# Assign trace_id as the index
df_timestamps = df_timestamps.set_index('trace_id')

# Restric df_timestamps to the same_tpl_meta indices
df_timestamps = df_timestamps[df_timestamps.index.isin(same_tpl_meta.index)]

df_timestamps

,@timestamp
trace_id,
1,2019-04-10 00:20:35.848
2,2019-04-10 00:42:17.106
3,2019-04-10 01:11:09.325
5,2019-04-10 01:29:34.143
6,2019-04-10 01:42:20.968
7,2019-04-10 02:05:36.920
10,2019-04-10 02:34:37.587
11,2019-04-10 03:01:37.165
12,2019-04-10 03:26:54.570


In [12]:
# Iterate over the indices of the rows in the meta dataframe and store the params in a list
# Extract only when numval is not NaN
# Generates a dictionary with timestamp as key, and each param as a column

params_dict = {}

for index in same_tpl_meta.index:
    # Load the trace
    df = df_traces[df_traces['trace_id'] == index]
    # Extract the params    
    params = extract_params(df)
    # Remove NaN values in numval
    params = params[params['numval'].notna()]
    # Remove strval
    params = params.drop(columns=['strval'])
    # Add the params dataframe to the dictionary
    params_dict[df_timestamps.loc[index, '@timestamp']] = params

first_key = list(params_dict.keys())[0]

# Display the first key
print(first_key)
params_dict[first_key]

2019-04-10 00:20:35.848000


,numval
param,
DET1.APOTYPE.VAL,0.000
DET1.APOX.VAL,512.000
DET1.APOY.VAL,512.000
DET1.DIT,0.111
DET1.INTERFX.VAL,512.000
DET1.INTERFY.VAL,512.000
DET1.NCOHERENT.VAL,1.000
DET1.NINCOHERENT.VAL,1.000
DET1.SNRLIMIT.VAL,5.000


In [13]:
# Now we need to be sure that all params in every element of params_dict are the considered.
# We will create a set of all params in all elements of params_dict
all_params = set()
for params in params_dict.values():
    all_params.update(params.index)
all_params = sorted(all_params)
all_params

['DET1.APOTYPE.VAL',
 'DET1.APOX.VAL',
 'DET1.APOY.VAL',
 'DET1.DIT',
 'DET1.INTERFX.VAL',
 'DET1.INTERFY.VAL',
 'DET1.NCOHERENT.VAL',
 'DET1.NINCOHERENT.VAL',
 'DET1.SNRLIMIT.VAL',
 'DET1.WLMAXCUT.VAL',
 'DET1.WLMINCUT.VAL',
 'DET2.APOTYPE.VAL',
 'DET2.APOX.VAL',
 'DET2.APOY.VAL',
 'DET2.INTERFX.VAL',
 'DET2.INTERFY.VAL',
 'DET2.NCOHERENT.VAL',
 'DET2.NINCOHERENT.VAL',
 'DET2.SNRLIMIT.VAL',
 'DET2.WLMAXCUT.VAL',
 'DET2.WLMINCUT.VAL',
 'INS.SFL.NAME',
 'INS.SFN.NAME',
 'SEQ.DIL.WL0',
 'SEQ.FRINGES.DURATION',
 'SEQ.FRINGES.NCYCLES',
 'SEQ.OPDM.L.WLMAX',
 'SEQ.OPDM.N.WLMAX',
 'SEQ.PHOTO.DURATION',
 'SEQ.SKY.DURATION',
 'SEQ.SKY.OFFS.ALPHA',
 'SEQ.SKY.OFFS.DELTA',
 'TEL.CHOP.FREQ',
 'TEL.CHOP.POSANG',
 'TEL.CHOP.PVRATIO',
 'TEL.CHOP.THROW']

In [14]:
# Add the missing elements in params_dict of all_params to each element of params_dict, if corresponds

# Create a new dictionary with the same keys as params_dict
# and the same values as params_dict, but with all_params as index
params_dict_complete = {}   

for key, params in params_dict.items():
    # Create a new dataframe with all_params as index
    params_complete = pd.DataFrame(index=all_params)
    # Add the params dataframe to the new dataframe
    params_complete = params_complete.join(params)
    # Add the new dataframe to the dictionary
    params_dict_complete[key] = params_complete
    
params_dict_complete[first_key]

,numval
DET1.APOTYPE.VAL,0.000
DET1.APOX.VAL,512.000
DET1.APOY.VAL,512.000
DET1.DIT,0.111
DET1.INTERFX.VAL,512.000
DET1.INTERFY.VAL,512.000
DET1.NCOHERENT.VAL,1.000
DET1.NINCOHERENT.VAL,1.000
DET1.SNRLIMIT.VAL,5.000
DET1.WLMAXCUT.VAL,4.800


In [15]:
# Finally, from params_dict_complete which has the form timestamp -> params,
# we will create a new dataframe with the timeseries for every param name.
# This new dataframe will have the index as the parameter name, and the columns as the timestamps.

params_df = pd.DataFrame(index=all_params)
for key, params in params_dict_complete.items():
    # Add the params dataframe to the new dataframe
    params_df[key] = params['numval']
    
# But is more useful to have it trasposed for better analysis

params_df.T

,DET1.APOTYPE.VAL,DET1.APOX.VAL,DET1.APOY.VAL,DET1.DIT,DET1.INTERFX.VAL,DET1.INTERFY.VAL,DET1.NCOHERENT.VAL,DET1.NINCOHERENT.VAL,DET1.SNRLIMIT.VAL,DET1.WLMAXCUT.VAL,...,SEQ.OPDM.L.WLMAX,SEQ.OPDM.N.WLMAX,SEQ.PHOTO.DURATION,SEQ.SKY.DURATION,SEQ.SKY.OFFS.ALPHA,SEQ.SKY.OFFS.DELTA,TEL.CHOP.FREQ,TEL.CHOP.POSANG,TEL.CHOP.PVRATIO,TEL.CHOP.THROW
2019-04-10 00:20:35.848,0.0,512.0,512.0,0.111,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,3.0
2019-04-10 00:42:17.106,0.0,512.0,512.0,0.060,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,3.0
2019-04-10 01:11:09.325,0.0,512.0,512.0,0.075,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,3.0
2019-04-10 01:29:34.143,0.0,512.0,512.0,0.075,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,4.5
2019-04-10 01:42:20.968,0.0,512.0,512.0,0.060,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,4.5
2019-04-10 02:05:36.920,0.0,512.0,512.0,0.111,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,3.0
2019-04-10 02:34:37.587,0.0,512.0,512.0,0.111,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,3.0
2019-04-10 03:01:37.165,0.0,512.0,512.0,0.060,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,3.0
2019-04-10 03:26:54.570,0.0,512.0,512.0,0.075,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,3.0
2019-04-10 04:00:48.024,0.0,512.0,512.0,0.075,512.0,512.0,1.0,1.0,5.0,4.8,...,4.0,13.0,60.0,30.0,1.0,15.0,0.5,0.0,1.0,3.0


## Recap: Create a params_df

In [16]:
def create_params_df(TPL_ID, df_meta, df_traces):
    same_tpl_meta = df_meta[df_meta['TPL_ID'] == TPL_ID]

    df_timestamps = df_traces.groupby('trace_id').first().reset_index()[['trace_id', '@timestamp']]
    df_timestamps = df_timestamps.set_index('trace_id')
    df_timestamps = df_timestamps[df_timestamps.index.isin(same_tpl_meta.index)]

    params_dict = {}

    for index in same_tpl_meta.index:
        df = df_traces[df_traces['trace_id'] == index]
        params = extract_params(df)
        params = params[params['numval'].notna()]
        params = params.drop(columns=['strval'])
        params_dict[df_timestamps.loc[index, '@timestamp']] = params    
        
    all_params = set()
    for params in params_dict.values():
        all_params.update(params.index)
    all_params = sorted(all_params)

    params_dict_complete = {}
    for key, params in params_dict.items():
        params_complete = pd.DataFrame(index=all_params)
        params_complete = params_complete.join(params)
        params_dict_complete[key] = params_complete

    params_df = pd.DataFrame(index=all_params)
    for key, params in params_dict_complete.items():
        params_df[key] = params['numval']

    return params_df.T


### Test for one bigger dataset

In [21]:
# PIONIER MATISSE GRAVITY, 1d 1w 1m 6m
INSTRUMENT='MATISSE'
RANGE='1w'
logs = ParlogsObservations(period=RANGE, source="Instrument", system=INSTRUMENT)

df_meta = logs.meta
df_traces = logs.traces()

TPL_IDs = logs.meta['TPL_ID'].value_counts()
TPL_ID= TPL_IDs.index[0]

params_df = create_params_df(TPL_ID, df_meta, df_traces)

print(f"For Instrument {INSTRUMENT} in range={RANGE} and TPL_ID {TPL_ID}, the params dataframe is:")

with pd.option_context('display.max_columns', None):
    display( params_df.describe() )

For Instrument MATISSE in range=1w and TPL_ID MATISSE_img_acq, the params dataframe is:


,COU.AG.ALPHA,COU.AG.DELTA,COU.AG.EPOCH,COU.AG.EQUINOX,COU.AG.PMA,COU.AG.PMD,COU.GS.MAG,DEL.REF.OPL,DET1.APOTYPE.VAL,DET1.APOX.VAL,DET1.APOY.VAL,DET1.DIT,DET1.INTERFX.VAL,DET1.INTERFY.VAL,DET1.NCOHERENT.VAL,DET1.NDIT,DET1.NINCOHERENT.VAL,DET1.SNRLIMIT.VAL,DET1.WLMAXCUT.VAL,DET1.WLMINCUT.VAL,DET2.APOTYPE.VAL,DET2.APOX.VAL,DET2.APOY.VAL,DET2.DIT,DET2.INTERFX.VAL,DET2.INTERFY.VAL,DET2.NCOHERENT.VAL,DET2.NDIT,DET2.NINCOHERENT.VAL,DET2.SNRLIMIT.VAL,DET2.WLMAXCUT.VAL,DET2.WLMINCUT.VAL,GetAutoLADC.-.max.ODP,INS.SFL.NAME,INS.SFN.NAME,SEQ.ACQ.DET1.DIT,SEQ.ACQ.DET2.DIT,SEQ.ACQ.SKY.DURATION,SEQ.ACQ.TARG.DURATION,SEQ.DIL.WL0,SEQ.FS.DET1.DIT,SEQ.FS.DET2.DIT,SEQ.FS.INS.SFL.NAME,SEQ.FS.INS.SFN.NAME,SEQ.FS.SKY.DURATION,SEQ.FS.TARG.DURATION,SEQ.OPDM.L.WLMAX,SEQ.OPDM.N.WLMAX,SEQ.OPDM.NSTEPS,SEQ.PUP.DET1.DIT,SEQ.PUP.SKY.DURATION,SEQ.PUP.TARG.DURATION,SEQ.SKY.OFFS.ALPHA,SEQ.SKY.OFFS.DELTA,SEQ.TARG.FLUX.L,SEQ.TARG.FLUX.N,SEQ.TARG.MAG.K,TEL.CHOP.FREQ,TEL.CHOP.POSANG,TEL.CHOP.PVRATIO,TEL.CHOP.THROW,TEL.GS1.ALPHA,TEL.GS1.DELTA,TEL.GS1.MAG,TEL.TARG.ADDVELALPHA,TEL.TARG.ADDVELDELTA,TEL.TARG.ALPHA,TEL.TARG.DELTA,TEL.TARG.EQUINOX,TEL.TARG.PMA,TEL.TARG.PMD
count,42.0,42.0,42.0,42.0,42.0,42.0,42.000000,42.0,43.0,43.0,43.0,1.000,43.0,43.0,43.0,1.0,43.0,43.0,43.000000,43.0,43.0,43.0,43.0,1.00,43.0,43.0,43.0,1.0,43.0,43.0,43.0,43.0,26.000000,1.0,1.0,42.000,42.000,42.0,42.0,43.0,42.000,1.00,42.0,42.0,42.0,42.0,42.0,42.0,1.0,42.00,42.0,42.0,42.0,42.0,42.000000,42.000000,42.000000,42.0,42.0,42.0,42.0,42.0,42.0,42.0,42.0,42.0,42.000000,42.000000,42.0,42.000000,42.000000
mean,0.0,0.0,2000.0,2000.0,0.0,0.0,2.377738,30.0,0.0,512.0,512.0,0.011,512.0,512.0,1.0,40.0,1.0,5.0,4.804651,3.0,0.0,512.0,512.0,0.02,512.0,512.0,20.0,40.0,1.0,5.0,10.0,8.0,0.000004,1.5,2.0,0.005,0.005,10.0,10.0,3.5,0.111,0.02,1.5,2.0,15.0,30.0,4.0,13.0,10.0,0.06,10.0,10.0,1.0,15.0,1215.809524,212.056190,-0.079738,0.5,0.0,1.0,4.5,0.0,0.0,12.0,0.0,0.0,102776.628143,-352836.491429,2000.0,83.230591,-0.227080
std,0.0,0.0,0.0,0.0,0.0,0.0,3.135278,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.030500,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.000002,NaN,NaN,0.000,0.000,0.0,0.0,0.0,0.000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.00,0.0,0.0,0.0,0.0,1315.545341,230.307965,1.131102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22749.154118,145459.780602,0.0,119.353383,0.397234
min,0.0,0.0,2000.0,2000.0,0.0,0.0,0.000000,30.0,0.0,512.0,512.0,0.011,512.0,512.0,1.0,40.0,1.0,5.0,4.800000,3.0,0.0,512.0,512.0,0.02,512.0,512.0,20.0,40.0,1.0,5.0,10.0,8.0,0.000001,1.5,2.0,0.005,0.005,10.0,10.0,3.5,0.111,0.02,1.5,2.0,15.0,30.0,4.0,13.0,10.0,0.06,10.0,10.0,1.0,15.0,1.600000,0.100000,-1.660000,0.5,0.0,1.0,4.5,0.0,0.0,12.0,0.0,0.0,64508.910000,-570647.560000,2000.0,-0.546000,-1.220000
25%,0.0,0.0,2000.0,2000.0,0.0,0.0,0.000000,30.0,0.0,512.0,512.0,0.011,512.0,512.0,1.0,40.0,1.0,5.0,4.800000,3.0,0.0,512.0,512.0,0.02,512.0,512.0,20.0,40.0,1.0,5.0,10.0,8.0,0.000003,1.5,2.0,0.005,0.005,10.0,10.0,3.5,0.111,0.02,1.5,2.0,15.0,30.0,4.0,13.0,10.0,0.06,10.0,10.0,1.0,15.0,65.500000,16.400000,-1.080750,0.5,0.0,1.0,4.5,0.0,0.0,12.0,0.0,0.0,100000.000000,-450000.000000,2000.0,-0.041700,-0.265000
50%,0.0,0.0,2000.0,2000.0,0.0,0.0,0.820000,30.0,0.0,512.0,512.0,0.011,512.0,512.0,1.0,40.0,1.0,5.0,4.800000,3.0,0.0,512.0,512.0,0.02,512.0,512.0,20.0,40.0,1.0,5.0,10.0,8.0,0.000003,1.5,2.0,0.005,0.005,10.0,10.0,3.5,0.111,0.02,1.5,2.0,15.0,30.0,4.0,13.0,10.0,0.06,10.0,10.0,1.0,15.0,700.000000,143.000000,0.000000,0.5,0.0,1.0,4.5,0.0,0.0,12.0,0.0,0.0,100000.000000,-370815.900000,2000.0,0.028230,-0.032382
75%,0.0,0.0,2000.0,2000.0,0.0,0.0,4.600000,30.0,0.0,512.0,512.0,0.011,512.0,512.0,1.0,40.0,1.0,5.0,4.800000,3.0,0.0,512.0,512.0,0.02,512.0,512.0,20.0,40.0,1.0,5.0,10.0,8.0,0.000005,1.5,2.0,0.005,0.005,10.0,10.0,3.5,0.111,0.02,1.5,2.0,15.0,30.0,4.0,13.0,10.0,0.06,10.0,10.0,1.0,15.0,3000.000000,300.000000,0.000000,0.5,0.0,1.0,4.5,0.0,0.0,12.0,0.0,0.0,112436.590000,-342702.700000,2000.0,250.000000,0.000000
max,0.0,0.0,2000.0,2000.0,0.0,0.0,9.800000,30.0,0.0

## Comments

* The functions extract_params and create_params_df needs more testing and validation. I tried with MATISSE/1m and PIONIER/1m and they failed.
* The sample analysis shown above covers only numerical values. The categorical values in column `strval` should be analyzed.
* I need a way to discover which columns changes to focus the analysis on those, and discard the fixed ones
* Another kind of analysis would be to discover which combination of parameters could end in ERROR=True.

*-- JPG, 2025-04-27*

## References

The parlogs-observations dataset are linked with specific periods and instruments. The PDF of each instrument linked below applies exactly to the dataset used in this notebook.

* GRAVITY
  * https://www.eso.org/sci/facilities/paranal/instruments/gravity.html
  * https://www.eso.org/sci/facilities/paranal/instruments/gravity/doc/GRAVITY_UserManual_P104.pdf
  * https://www.eso.org/sci/facilities/paranal/instruments/gravity/doc/GRAVITY_TemplateManual_P104.pdf
* MATISSE
  * https://www.eso.org/sci/facilities/paranal/instruments/matisse.html
  * http://www.eso.org/sci/facilities/paranal/instruments/matisse/doc/VLT-MAN-ESO-289268_P103.pdf
  * http://www.eso.org/sci/facilities/paranal/instruments/matisse/doc/ESO-255467_P103.pdf
* PIONIER
  * https://www.eso.org/sci/facilities/paranal/instruments/pionier.html
  * https://www.eso.org/sci/facilities/paranal/instruments/pionier/doc/VLT-MAN-ESO-263601_v102.pdf
  * https://www.eso.org/sci/facilities/paranal/instruments/pionier/doc/VLT-MAN-ESO-270590_v102.pdf
* Parlogs-observations
  * https://huggingface.co/datasets/Paranal/parlogs-observations
  * https://github.com/Paranal-SW/parlogs-observations